# Importing Packages

In [2]:
import pandas as pd
import numpy as np

# Loading Files and Defining Vars and Params


In [7]:
# loading items
items = pd.read_csv('items.csv', usecols = [1, 2, 3], encoding = "ISO-8859-1")

# loading trasactions
transactions = pd.read_csv('transactions.csv', usecols = [1])

# drop useless column
transactions = transactions.rename(columns= {'Col0':'Items'})
transactions.index.name = 'Basket'

# defining variables
num_items = len(items)
num_basket = len(transactions)
threshold_s = int(0.1 * len(transactions))

# showing datas
print("items :")
display(items.head())
print("transactions : ")
display(transactions.head())

items :


,Item,Price,ProductName
0,0,2.511933,swiss cheese
1,1,52.441562,Cherry coke
2,2,28.215382,Bio Coke
3,3,3.991994,Peppers
4,4,1.042489,scrambled egg


transactions : 


,Items
Basket,
0,224 80 109 177 50 43 83 173 70 202 94 227 162 ...
1,56 95 106 186 103 170 69 198 186 211 83 24 78 ...
2,9 196 184 119 88 196 222 94 212 187 95 3 224 5...
3,228 9 193 127 163 117 24 34 204 163 48 74 69 2...
4,94 9 22 133 107 228 77 173 38 109 32 31 110 79...


# Part A ) Finding single frequent items

first we make a list out of each basket. Then we convert all basket to one hot, so that each column of dataframe represent existence of a item in baskets. Now on first step of apriori we should find single frequent items. So we sum on each column to count number of baskets each item is in. Then we filter to get frequent items.

In [8]:
def one_hot_func(row):
    vector = np.zeros((num_items))
    for item in row:
        vector[item] = 1
        
    return vector

 
# making each basket from string to list of items
transactions = transactions.applymap(lambda x: [int(y) for y in x.split(' ')])

# one hot all columns
one_hot_trans = transactions.applymap(one_hot_func)

# sum on every column so that we get count of each item
sum_col = one_hot_trans.apply(sum)

# build a dataframe with columns ["item", "count"]
sum_col = sum_col.reset_index().rename(columns = {'index':'item_id', 'Items':'count'})

# filtering base on support so we get single frequent item
L1 = sum_col[sum_col['count'] >= threshold_s]
L1 = L1.reset_index(drop = True)

print("some of single frequent items: ")
display(L1.head())

some of single frequent items: 


,item_id,count
0,2,403.0
1,12,919.0
2,17,319.0
3,32,312.0
4,33,325.0


# Continue of Apriori

I explained in details with comment but for summary, in each round first we compute new candidates base on previous candidates and single frequent items. But we update list of single freqs on each step, so our space of search don't grow too much. For example to make candidates of round 3, we don't use all of single freqs, we use single freqs which exists in doubleton frequent items. Then we count all candidates and save it in dataframe. and in last we filter them by support.

In [9]:
# building candidate of next round base candidate of previous round and updated single freq items
def new_candidates(last_freq):
    out_set = []
    
    for item in last_freq:
        for freq_i in freq_item:
            if(freq_i not in item):
                out_set.append(tuple(sorted(item + (freq_i, ))))
            
    out_set = list(set(out_set))
    
    return out_set

# check if candidate tuples exist in basket
def match_item(basket):
    
    for freq_i in ck:
        if(set(freq_i).issubset(basket)):
            tuple_count.append((freq_i, 1))


# build a one hot vector
def one_vector(list_item):
    
    one_hot = np.zeros((num_items))
    
    for item in list_item:
        one_hot[item] = 1
        
    return one_hot

# check if candidate tuples exist in basket with one hot method
def match_one(basket):
    
    for freq_i in ck:
        freq_one = one_vector(set(freq_i))
        if(np.dot(freq_one, basket) == k ):
            tuple_count.append((freq_i, 1))
            

# update items for creating candidates of next round
def update_freque_item():
    unique_set = []
    for item in Lk['item_id']:
        unique_set.extend([y for y in item])
        unique_set = list(set(unique_set))
    return unique_set
    
# to hold all of frequent items base on their len
freq_item_dict = dict()

# showing single frequents
print("1 element frequent items:")
display(L1)

# candidates for next round is items of single frequent
ck = [(item,) for item in L1['item_id']]
freq_item_dict[1] = L1

# list of single freq items to build next candidate
freq_item = L1['item_id']

# k is showing size of frequent items
k = 2
# list holding freq itemsets of len K
k_freq_item = [1]

while(len(k_freq_item) != 0): # continue till list is empty
    
    # building new candidates base on candidates of previous round
    ck = new_candidates(ck)
    
    # holding count of each candidate tuple
    tuple_count = []
    
    # counting base on one hot method
    #one_hot_trans.applymap(match_one)
    
    # counting base on default df ( this method is faster )
    for row in transactions['Items']:
        match_item(row)
    
    # building dataframe with columns = ["candidate tuples", "count"] and sum all equal tuples
    k_frequent = pd.DataFrame(data = tuple_count, columns = ['item_id', 'count']).groupby('item_id').agg({'count':'sum'})
    
    # filtering base on support
    Lk = k_frequent[k_frequent['count'] >= threshold_s]
    Lk = Lk.reset_index()
    Lk = Lk.sort_values(by='item_id')
    
    # displaying result of each round
    if( len(Lk) !=0 ):
        
        print(str(k) + " elements frequent items:")
        display(Lk)
        
    k_freq_item = Lk['item_id']
    if(len(k_freq_item) != 0):
        freq_item_dict[k] = Lk
        print("len of " + str(k) + " elements frequent is = " + str(len(k_freq_item)))
    
    # go to next round
    k = k + 1
    
    # delete all unfrequent items from comparing ck and ck-1
    freq_item = update_freque_item()
    print("len of new freq items = " + str(len(freq_item)))
    
    
    


1 element frequent items:


,item_id,count
0,2,403.0
1,12,919.0
2,17,319.0
3,32,312.0
4,33,325.0
5,36,329.0
6,58,300.0
7,60,376.0
8,62,337.0
9,74,379.0


2 elements frequent items:


,item_id,count
0,"(12, 117)",333
1,"(12, 235)",919
2,"(58, 91)",300
3,"(117, 235)",333


len of 2 elements frequent is = 4
len of new freq items = 5
3 elements frequent items:


,item_id,count
0,"(12, 117, 235)",333


len of 3 elements frequent is = 1
len of new freq items = 3
len of new freq items = 0


# Pack all frequents and save them

In [10]:
# holding all frequent items
all_freq = pd.DataFrame()

for val in freq_item_dict:
    all_freq = all_freq.append(freq_item_dict[val])
    
all_freq.to_csv('freq_itemsets_10.csv')

# Part B ) Finding Association Rules and Parameters

To find rules. First we consider freq itemsets of len more than 1. Then for each itemsets we find subsets. For example for set = (1,2,3), we find subsets = {1,2,3,(1,2),(1,3),(2,3)}. and then we can build rules with this. We consider each subset as antecedent and ( set - subset ) as consequent. Then we compute strength of these rules base on statistical parameters like confidence and interest and conviction. Then we sort rules base on interest so we can find important rules.

In [14]:
from itertools import combinations

# finding all subset of lists with len = [1,n)
def subset(setInp):
    subsets = []
    for i in np.arange(1, len(setInp)):
        subsets.extend(list(combinations(setInp, i)))
    return subsets

def index_builder(tuple_ind):
    return tuple_ind[0] if len(tuple_ind) == 1 else tuple_ind

def association_rule(row):
    # holding rules
    confidence_freq = pd.DataFrame(columns = ["antecedents", "consequents", "confidence", "interest", "conviction"])
    
    # finding all subset of freq tuples
    tuple_freq = row[0]
    sub_freq = subset(tuple_freq)
    
    for sub in sub_freq:
        # finding consequents
        diff = tuple(sorted(set(tuple_freq) - set(sub)))
        
        # finding index of antecedents and consequents in dataframe of freq itemsets
        indx_sub = index_builder(sub)
        indx_diff = index_builder(diff)
        
        # finding support of antecedents and consequents and their join
        supp_sub = int(all_freq_reindex.loc[[indx_sub]]['count'])
        supp_tuple = int(all_freq_reindex.loc[[tuple_freq]]['count'])
        prob_diff = int(all_freq_reindex.loc[[tuple_freq]]['count']) / len_basket
        
        # computing confidence and interest and conviction
        confidence = supp_tuple / supp_sub
        interest = np.abs(confidence - prob_diff)
        conviction = np.float64(1 - prob_diff) / (1 - confidence)
        
        
        # filter base on confidence
        if ( confidence >= conf_treshold):
            confidence_freq = confidence_freq.append({'antecedents':sub, 'consequents':diff,
                                                      'confidence':confidence, 'interest':interest,
                                                     'conviction': conviction}, ignore_index=True)
            
           
    return  confidence_freq   
        
    
    
    
# confidence treshold   
conf_treshold = 0.1

# len of baskets
len_basket = len(transactions)

# we want freq itemsets with len more than one to find rules
multi_freq = all_freq[all_freq['item_id'].apply(type) == type(())]

# changing freq items df to make accessability faster
all_freq_reindex = all_freq.set_index('item_id')

# output df of rules
confidence_freq = pd.DataFrame(columns = ["antecedents", "consequents", "confidence", "interest", "conviction"])

for i in range(len(multi_freq)):
    confidence_freq = confidence_freq.append(association_rule(multi_freq.iloc[i][['item_id','count']]))

# sort base on interest of rules
confidence_freq = confidence_freq.sort_values("interest", ascending= False)

# showing some of rules
display(confidence_freq.head(10))

<ipython-input-14-72ce4f8c5e0c>:37: RuntimeWarning: divide by zero encountered in double_scalars
  conviction = np.float64(1 - prob_diff) / (1 - confidence)


,antecedents,consequents,confidence,interest,conviction
0,"(58,)","(91,)",1.000000,0.895434,inf
1,"(91,)","(58,)",1.000000,0.895434,inf
3,"(12, 117)","(235,)",1.000000,0.883932,inf
5,"(117, 235)","(12,)",1.000000,0.883932,inf
0,"(12,)","(235,)",1.000000,0.679679,inf
1,"(235,)","(12,)",1.000000,0.679679,inf
1,"(117,)","(12,)",0.414695,0.298627,1.510207
0,"(117,)","(235,)",0.414695,0.298627,1.510207
1,"(117,)","(12, 235)",0.414695,0.298627,1.510207
0,"(12,)","(117,)",0.362350,0.246282,1.386234


# Make rules readable

In [26]:
# convert tuples of item ids into item names
def id_name(tuple_item):
    list_item = list(tuple_item)
    out_tuple = []
    
    for item in list_item:
            out_tuple.append(items.loc[item]['ProductName'])
        
    return tuple(out_tuple)

# iterate on column of df
def column_builder(col):
    out_col = []
    
    for item in col:
        out_col.append(id_name(item))
    
    return out_col

# make rules readable
name_rule_df = confidence_freq.apply(lambda x: column_builder(x) if x.name in ['antecedents','consequents']  else x)

print("some of strong rules :")
display(name_rule_df)

some of strong rules :


,antecedents,consequents,confidence,interest,conviction
0,"(Watermelon,)","(Watermelon,)",1.000000,0.895434,inf
1,"(Watermelon,)","(Watermelon,)",1.000000,0.895434,inf
3,"(shrimps, Tuna)","(lobster,)",1.000000,0.883932,inf
5,"(Tuna, lobster)","(shrimps,)",1.000000,0.883932,inf
0,"(shrimps,)","(lobster,)",1.000000,0.679679,inf
1,"(lobster,)","(shrimps,)",1.000000,0.679679,inf
1,"(Tuna,)","(shrimps,)",0.414695,0.298627,1.510207
0,"(Tuna,)","(lobster,)",0.414695,0.298627,1.510207
1,"(Tuna,)","(shrimps, lobster)",0.414695,0.298627,1.510207
0,"(shrimps,)","(Tuna,)",0.362350,0.246282,1.386234


# Part J ) Suggestion of new items to Buy for given Basket

We use rule with interest more than 0.5 with trial and error, others were kina meaningless. To find suggestions, first we find subsets of given basket. Now for each subset we search for rule so that subset is equal to  antecedent. Then we give them suggestions using that rules.

In [24]:
# we use only rule which hase interest more than 0.5, i check this value by trial and error
interest_freq  = confidence_freq[ confidence_freq['interest'] >= 0.5]

# make list of antecedents
antecedents_list = list(interest_freq['antecedents'])

# our given basket
our_basket = (12,21,34,123,134)
our_basket_name = id_name(our_basket)

# finding subsets of baskets
sub_basket = subset(our_basket)

# showing basket
print("for basket :")
print(our_basket_name)

# showing suggestions
print("We have these suggestions : ")

# search on subsets of basket to see if there is any rule for it
for sub in sub_basket:
    # convert tuples of ids to tuple of names
    sub_name = id_name(sub)
    
    if(sub in antecedents_list):
        # find rules with subset of basket
        matched_rows = interest_freq[interest_freq['antecedents'] == sub]
        
        # for found rules shows suggestions
        for suggest in list(matched_rows['consequents']):
            
            print("Because of " + str(sub_name) + " in basket, we suggest " +
                  str(id_name(suggest)))
        
        


for basket :
('shrimps', 'cucumber', 'mussels', 'Bio lemonade', 'cookies')
We have these suggestions : 
Because of ('shrimps',) in basket, we suggest ('lobster',)
